# Clase 3


Objetivos:

1. Simulación de flujos metabólicos usando programación lineal
2. Implementación de problemas de programación lineal con numpy y scipy

## Simulación de flujos metabólicos usando Programación lineal

La distribución de flujos metabolicos de una celula puede ser modelado como:

\begin{align}
\mbox{max}\ & f(x)=c^Tx \\
\mbox{s.a.} & \\
&Sx=0 \\
&LB<=x<=UB
\end{align}

En donde `S` es la matriz estequiométrica, `x` los flujos metabólicos, y `LB` y `UB` que son los limites inferiores (lower bound) y superiores (upper bound) de cada flujo. Por otra parte, `c` contiene los coeficientes que ponderan el aporte de cada `x` a la generación de una función apropiada, las cuales tipicamente corresponden a biomasa o producción de energía. 

Para resolver este tipo de problemas usando el método simplex existe la librería `scipy.optimize.linprog`. Típicamente un modelo metabólico consiste en restricciones de igualdad, por ejemplo: 


\begin{align}
\mbox{min}\ &f(x)=70x_1+80X_2+85X_3 \\
\mbox{s.a:}\ & \\
&x_1+x_2+x_3+x_4=999 \\
&x_1+4x_2+8x_3+x_5 = 4500 \\
&40x_1 + 30x_2 +20x_3 = 36000 \\
&3x_1 + 2x_2 +4x_3 +x_6= 2700 \\
&x>=0
\end{align}

puede ser resuelto con la siguiente código:

In [1]:
import numpy as np
from scipy.optimize import linprog
from numpy.linalg import solve

A = np.array([
[1, 1, 1, 0, 0, 0],
[1, 4, 8, 1, 0, 0],
[40, 30, 20, 0, 1, 0],
[3, 2, 4, 0, 0, 1]])

b = np.array([999, 4500, 36000, 2700])
c = np.array([70, 80, 85, 0, 0, 0])

res = linprog(c, A_eq=A, b_eq=b, bounds=(0, None))
print('Optimal value:', res.fun, '\nX:', res.x)

Optimal value: 73724.98976093884 
X: [6.35999906e+02 3.29999957e+02 3.29999978e+01 2.27999966e+03
 7.23524164e-05 4.88371031e-07]


Sin embargo, `linprog` tambien nos permite resolver problemas en donde las restricciones son una mezcla de igualdades y desigualdades. 

Por ejemplo:

\begin{align}
\mbox{min}\ &f(x)=70x_1+80X_2+85X_3 \\
\mbox{s.a:}\ & \\
&x_1+x_2+x_3=999 \\
&x_1+4x_2+8x_3 \le 4500 \\
&40x_1 + 30x_2 +20x_3 \le 36000 \\
&3x_1 + 2x_2 +4x_3 \le 2700 \\
&x>=0
\end{align}

Puede ser resuelto con las siguiente sintaxis:

In [2]:
import numpy as np
from scipy.optimize import linprog
from numpy.linalg import solve

A_eq = np.array([[1,1,1]])
b_eq = np.array([999])

A_ub = np.array([
[1, 4, 8],
[40,30,20],
[3,2,4]])

b_ub = np.array([4500, 36000,2700])

c = np.array([70, 80, 85])

res = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub,
bounds=(0, None))
print('Optimal value:', res.fun, '\nX:', res.x)

Optimal value: 73724.98976090914 
X: [635.99990607 329.99995653  32.99999781]


## Ejercicio 1


Escribe una función para resolver simular los flujos de la siguiente red metabólica.

## Ejemplo aplicado a redes metabólicas

Ok, ahora interpretemos los $x$s, $S$s, y otros vectores y estructuras matemáticas a la luz de una pregunta biológicamente significativa: ¿cuál es el origen de la vida? 

Para eso tenemos que tener una definición de "vida". Al respecto la NASA la define como: ["Un sistema químico autosustentable capaz de evolución Darwiniana"](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3005285/). Esta definición implica que los objetos vivos tienen materical genético a sus disposición (evolución Darwiniana), el cual dificilmente se crea de manera espontanea. 

Una explicación a la aparición de bases nucleotidas es la hipotesis del ["metabolismo primero"](http://www.preposterousuniverse.com/blog/2010/03/10/free-energy-and-the-meaning-of-life/). Según esta hipotesis, un conjunto de reacciones quimicas espontaneas pueden autoreplicarse para aumentar progresivamente la complejidad de las moleculas involucradas, las cuales a su vez pueden crear sistemas más complejos. Sólo cuando hay un nivel de complejidad mínimo se puede apelar al mecanismo de evolución Darwiniana. Veamos un ejemplo.

<img src="https://raw.githubusercontent.com/modcommet/Clases/master/methaneFreeEnergy.jpg" width="400">
Perfil de energía libre a través de la vía geoquímica (en purpura) a metano contrastado con la vías acetogénica bacteriana (triangulos) y metanogénica de las arqueas (cuadrados), ambas usando la via de la acetil conenzima-A.(Imagen adpatada de [Yung et al 2010](http://journalofcosmology.com/SearchForLife128.html) )


Usando la red de juguete de abajo, simula sus flujos.

<img src="https://raw.githubusercontent.com/modcommet/Clases/master/metanogenesis.png" width="400">

In [5]:
# Los flujos son v = [E1,E2,R1,R2,R3,E3,E4,E5]

import numpy as np
from scipy.optimize import linprog
from numpy.linalg import solve

S = np.array([
[1,0,-1,0,0,0,0,0], #CO2     
[0,2,-2,-2,0,0,0,0], #H2
[0,0,-1,1,1,0,-2,0], #F420
[0,0,1,-1,1,0,0,0], # I1
[0,0,0,1,-1,-1,0,0], #CH4
[0,0,0,0,2,0,0,-2]] )#H2O

b = np.array([0]*6)
c = np.array([0,0,0,0,0,1,0,0]) # Optimizamos la produccion de biomasa (metabol) a través del flujo E3 

LB = np.array([0]*8)
UB = np.array([1000]*8)
UB[0] =10 # Asumamos que la red metabólica está limitada por el consumo de CO2

res = linprog(-c, A_eq=S, b_eq=b, bounds=list(zip(LB, UB))  )
print('ctx:', res.fun, 'x:', res.x)

ctx: -9.999999999037586 x: [ 10.         528.34445193  10.         518.34445193 508.34445193
  10.         508.34445193 508.34445193]
